# Import libraries

In [ ]:
# Standard Library
import os
import json
from enum import Enum
from typing import Tuple, List
from pathlib import Path
# External library
from easydict import EasyDict
import cv2
import numpy as np
import wget
import matplotlib.pyplot as plt

In [2]:
!pip3 install opencv-python matplotlib numpy wget easydict


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Define constant

In [3]:
CUT_VALUE = 100
MIN_OBJECT_WIDTH = 800
MAX_OBJECT_WIDTH = 1500

# Define class

In [4]:
class DetectionResult(Enum):
    OK = 'OK'
    NG = 'NG'
    
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y


# Utils function

In [5]:
def adjust_contrast_brightness(img, contrast:float=1.0, brightness:int=0):
    """
    Adjusts contrast and brightness of an uint8 image.
    contrast:   (0.0,  inf) with 1.0 leaving the contrast as is
    brightness: [-255, 255] with 0 leaving the brightness as is
    """
    brightness += int(round(255*(1-contrast)/2))

    return cv2.addWeighted(img, contrast, img, 0, brightness)

In [6]:
def bounding_box_for_circles(circles: List) -> Tuple:
    """
    Calculate bounding box containing all the circles
    
    Args:
        circles (list): List of circles
    
    Returns:
        Tuple[Tuple[int, int], Tuple[int, int]]: Bounding box of the circles
    """
    # Example of circles: [{'point': Point(955, 566), 'radius': 11}, {'point': Point(955, 566), 'radius': 11}]
    x_min, x_max, y_min, y_max = [], [], [], []
    # Calculate min, max x, y with radius
    for circle in circles:
        x_min.append(circle['point'].x - circle['radius'])
        x_max.append(circle['point'].x + circle['radius'])
        y_min.append(circle['point'].y - circle['radius'])
        y_max.append(circle['point'].y + circle['radius'])
    
    # Calculate for bounding box 
    x_min, x_max, y_min, y_max = int(min(x_min)), int(max(x_max)), int(min(y_min)), int(max(y_max))
    # Define each corner of the bounding box
    A, B, C, D = (x_min, y_min), (x_max, y_min), (x_max, y_max), (x_min, y_max)
    
    # Store A, B, C, D in a dict
    bounding_box = {A:A, B:B, C:C, D:D}
    # With each corner find the nearest circle
    for corner in [A, B, C, D]:
        # Calculate distance between corner and circles
        distances = [np.linalg.norm(np.array(corner) - np.array((circle['point'].x, circle['point'].y))) for circle in circles]
        # Get the nearest circle
        nearest_circle = circles[np.argmin(distances)] # Example nearest_circle: {'point': Point(955, 566), 'radius': 11}
        # # Update corner A, B, C, D with the nearest circle
        bounding_box.update({corner: {'nearest_cirle_cX_cY': nearest_circle['point']}})
    
    return bounding_box[A], bounding_box[B], bounding_box[C], bounding_box[D]

In [7]:
def find_rectangle(image, min_object_width=0, max_object_width=9999, cut_value=100):
    """Find box of object

    Args:
        image (numpy.ndarray): image
        min_object_width (int): min value of object width
        max_object_width (int): max value of object width
        cut_value (int): cut into center

    Returns:
        x_tl, y_tl, x_br, y_br: x, y top left and bottom right
    """
    height, width = image.shape[:2]
    x_tl, y_tl, x_br, y_br = 0, 0, width, height
    try:
        image = adjust_contrast_brightness(image, 6.9, 170)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        _, binary = cv2.threshold(gray_image, 150, 255, cv2.THRESH_BINARY_INV)

        kernel = np.ones((3, 3), np.uint8)
        image_blur = cv2.GaussianBlur(binary, (5,5), 0)
        edges = cv2.Canny(image_blur, threshold1=100, threshold2=150)
        edges_dilated = cv2.dilate(edges, kernel, iterations=1)
        contours, _ = cv2.findContours(edges_dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        min_contour_area = 500
        large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]
        # find square
        largest_square = None
        max_area = 0

        for contour in large_contours:
            # square
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            
            if len(approx) == 4:
                area = cv2.contourArea(contour)
                if area > max_area:
                    largest_square = contour
                    max_area = area
        if largest_square is not None:
            x, y, w, h = cv2.boundingRect(largest_square)
            if min_object_width <= w <= max_object_width:
                x_tl, y_tl = x + cut_value, y + cut_value
                x_br, y_br = x + w - cut_value, y + h - cut_value

    except Exception as e:
        x_tl, y_tl, x_br, y_br = 0, 0, width, height

    return x_tl, y_tl, x_br, y_br

# Process

## Find the pins

In [23]:
def find_pin_circles(image_path: str, opts) -> List:
    """
    Find pin circles in the image
    
    Args:
        image_path (str): Path to the image
        
    Returns:
        np.ndarray: Detected pin circles
    """
    # Check if image exists
    
    if not os.path.exists(image_path):
        raise FileNotFoundError("Image not found")
    # Read image
    image = cv2.imread(image_path)
    # crop object
    scaled_image = cv2.resize(image, None, fx=opts.x_scale, fy=opts.y_scale, interpolation=cv2.INTER_LINEAR)
    x_tl, y_tl, x_br, y_br = find_rectangle(scaled_image, MIN_OBJECT_WIDTH, MAX_OBJECT_WIDTH, CUT_VALUE)
    scaled_image = scaled_image[y_tl:y_br, x_tl:x_br]

    # Adjust contrast and brightness
    scaled_image = adjust_contrast_brightness(scaled_image, 2.8, -250)

    # Convert to grayscale
    gray = cv2.cvtColor(scaled_image, cv2.COLOR_BGR2GRAY)
    # Find edges
    edges = cv2.Canny(gray, threshold1=50, threshold2=150)
    # Dilate edges
    kernel = np.ones((3, 3), np.uint8)
    edges_dilated = cv2.dilate(edges, kernel, iterations=1)
    # find and filter contours
    contours, _ = cv2.findContours(edges_dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 500]
    # find square
    largest_square = None
    max_area = 0
    corners = []
    for contour in large_contours:
        # square
        epsilon = 0.05 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        if len(approx) == 4:
            area = cv2.contourArea(contour)
            if area > max_area:
                largest_square = contour
                max_area = area
                corners = [tuple(point[0]) for point in approx]
    
    circles_near_corners = []
    if largest_square is not None:
        # remove largest square contour
        large_contours = [cnt for cnt in large_contours if not np.array_equal(cnt, largest_square)]
        # get bounding box of object
        x_square, y_square, w_square, h_square = cv2.boundingRect(largest_square)
        # for each corner, find a circle
        for corner in corners:
            found_circle = None
            min_distance = float('inf')
            for contour in large_contours:
                if cv2.arcLength(contour, True) > 0:
                    # find circle
                    (x, y), radius = cv2.minEnclosingCircle(contour)
                    center = (int(x), int(y))
                    radius = int(radius)
                    # roundness calculation
                    circularity = 4 * np.pi * cv2.contourArea(contour) / (cv2.arcLength(contour, True) ** 2)
                    if 0.5 <= circularity <= 1.5:
                        # if center inside bounding box of object then set circle
                        if (x_square <= center[0] <= x_square + w_square) and \
                            (y_square <= center[1] <= y_square + h_square):
                            
                            distance_to_corner = np.linalg.norm(np.array(center) - np.array(corner))
                            # find nearest circle
                            if distance_to_corner < min_distance:
                                min_distance = distance_to_corner
                                found_circle = contour
                                # find the center
                                moment = cv2.moments(contour)
                                cx = int(moment['m10'] / moment['m00'])
                                cy = int(moment['m01'] / moment['m00'])
                                circle_center = (cx, cy)
                                circle_radius = radius

            if found_circle is not None:
                circles_near_corners.append((found_circle, circle_center, circle_radius))
    # process circle
    circles_list = []
    for index, (contour, center, radius) in enumerate(circles_near_corners):
        point = Point(int(center[0] + x_tl), int(center[1] + y_tl))
        radius = radius
        circles_list.append({"point": point, "radius": radius})
    return circles_list

## Calculate the distance between the pins

In [9]:
def pair_distance_detection(circles: List, opts: EasyDict) -> str:
    """
    Detects the distance between the pairs of circles
    
    Args:
        circles (List): List of circles
        opts (EasyDict): dictionary of settings
        
    Returns:
        str: OK or NG
    """
    if circles is None or len(circles) < 4:
        print("Not enough circles detected")
        return DetectionResult.NG.value

    # Find bounding box for the detected circles
    A, B, C, D = bounding_box_for_circles(circles)
    
    # Define each adjacent vertices
    adjacent_vertices = [(A, B), (B, C), (C, D), (D, A)]
    
    # Calculate distance between the adjacent vertices
    for vertex1, vertex2 in adjacent_vertices:
        point1 = vertex1['nearest_cirle_cX_cY']
        point2 = vertex2['nearest_cirle_cX_cY']
        # Calculate distance between two points
        distance = np.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2) * opts.second_calibration
        distance = round(distance, opts.decimal_point)

        # Case the distance is not within the tolerance
        if round(abs(distance - opts.pin_distance), opts.decimal_point) > opts.pair_pin_distance_tolerance:

            return DetectionResult.NG.value
        
    return DetectionResult.OK.value

## Visualize

In [10]:
def visualize_result(image: np.ndarray, circles: List, opts: EasyDict) -> np.ndarray:
    """
    Visualize the result
    
    Args:
        image (np.ndarray): Image to visualize
        circles (List): Detected circles
        
    Returns:
        np.ndarray: Visualized image
        
    """
    for circle in circles:
        # Draw circles
        point = circle['point']
        radius = int(circle['radius']/opts.second_scale)
        new_point = Point(int(point.x/opts.second_scale), int(point.y/opts.second_scale))
        cv2.circle(image, (int(new_point.x), int(new_point.y)), radius, (0, 255, 0), 2)
        cv2.circle(image, (int(new_point.x), int(new_point.y)), 2, (0, 0, 255), 3)
        
    # Draw AB, BC, CD, DA
    A, B, C, D = bounding_box_for_circles(circles)
    # Adjacent vertices
    adjacent_vertices = [(A, B), (B, C), (C, D), (D, A)]
    distances = []
    # Draw lines and put distance text
    for vertex1, vertex2 in adjacent_vertices:
        point1 = vertex1['nearest_cirle_cX_cY']
        point2 = vertex2['nearest_cirle_cX_cY']
        distance = np.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2) * opts.second_calibration
        distance = round(distance, opts.decimal_point)
        distances.append(distance)
        new_point1 = Point(int(point1.x/opts.second_scale), int(point1.y/opts.second_scale))
        new_point2 = Point(int(point2.x/opts.second_scale), int(point2.y/opts.second_scale))

        cv2.line(image, (new_point1.x, new_point1.y), (new_point2.x, new_point2.y), (0, 255, 0), 2)
        cv2.putText(image, f"{distance:.{opts.decimal_point}f}mm", (int((new_point1.x + new_point2.x)/2), int((new_point1.y + new_point2.y)/2)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2)

    # Calculate the average distance
    average_distance = np.mean(distances)

    return average_distance, image

# Main Function

In [11]:
def measure_roundess_and_pair_distance_pins(input_path: str, output_path: str, opts: EasyDict) -> str:
    """
    Measure roundess and pair distance of pins
    
    Args:
        input_path (str): Path to the input image
        output_path (str): Path to the output image
        opts (EasyDict): Options
        
    Returns:
        str: Predicted class
    """
    visualized_image = cv2.imread(input_path)
    # Store predicted classes
    predict_classes = []
    try:
        # Find pin circles
        circles = find_pin_circles(input_path, opts)
        # Check if at least 4 circles are detected
        if len(circles) < 4:
            predict_classes.append(DetectionResult.NG.value)
            
        
        # Pair distance detection
        ok_ng = pair_distance_detection(circles, opts)
        predict_classes.append(ok_ng)
        
        # Visualize the result
        if opts.visualization:
            average_distance, visualized_image = visualize_result(visualized_image, circles, opts)
            
        # Save the output image
        if output_path:
            cv2.imwrite(output_path, visualized_image)
            
        # Final predict class
        final_predict_class = DetectionResult.OK.value if DetectionResult.NG.value not in predict_classes else DetectionResult.NG.value
    
    except Exception as e:
        print(f"Error: {e}")
        if output_path:
            cv2.imwrite(output_path, visualized_image)
        return 0, DetectionResult.NG.value
    
    return float(average_distance), final_predict_class

In [12]:
def run(input_path: str, output_path: str, opts) -> dict:
    """
    Execute third station
    
    Args:
        input_path (str): Path to the input image
        output_path (str): Path to the output image
        opts (EasyDict || Namespace): parameters for measurement
        
    Returns:
        dict: Result
    """

    average_distance, predict_class = measure_roundess_and_pair_distance_pins(input_path, output_path, opts)

    image = cv2.imread(input_path)
    height, width = image.shape[:2]

    result = {
        "file_name": os.path.basename(input_path),
        "predict_class": predict_class,
        "measurement_items": {
            "measurement_units": "px",
            "width": width,
            "height": height
        },
        "measurement_results": {
            "measurement_units": "mm",
            "pin_distance": average_distance
        },
        "comparison_parameters": {
            "measurement_units": "mm",
            "pin_distance": opts.pin_distance
        },
        "error": {
            "pin_distance": round(abs(opts.pin_distance - average_distance), opts.decimal_point)
        }
    }

    return result


# Run

In [ ]:
img_url = "https://ecos-ai-test-upload.s3.ap-northeast-1.amazonaws.com/data/alphavina/korea_electric/20241209114456_C2.jpeg"

input_path = r"C:\Users\Admin_PC\Desktop\robot\farino\input\only_pins.jpg"
#if not os.path.isfile(input_path):
    #wget.download(img_url, input_path)
opts = EasyDict({
    "x_scale": 1,
    "y_scale": 1
})
result = find_pin_circles(image_path=input_path, opts=opts)
print(result)

import cv2

image = cv2.imread(input_path)

for circle in result:
    point = circle.get("point")
    radius = circle.get("radius")

    if point is None or radius is None:
        print(f"⚠️ Skipping invalid entry: {circle}")
        continue

    # Access x and y from the Point object
    x, y = int(point.x), int(point.y)
    r = int(radius)

    # Draw the outer circle (green)
    cv2.circle(image, (x, y), r, (0, 255, 0), 2)
    # Draw the center point (red)
    cv2.circle(image, (x, y), 3, (0, 0, 255), -1)
    # Optional: label each circle
    cv2.putText(image, f"({x},{y}) r={r}", (x + 10, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 1)
cv2.imwrite(r"C:\Users\Admin_PC\Desktop\robot\farino\output\pins_visualized.jpg", image)
cv2.imshow("Detected Circles", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


[{'point': <__main__.Point object at 0x0000018C325563E0>, 'radius': 42}, {'point': <__main__.Point object at 0x0000018C32713B50>, 'radius': 28}, {'point': <__main__.Point object at 0x0000018C32807910>, 'radius': 29}, {'point': <__main__.Point object at 0x0000018C328067A0>, 'radius': 24}]
